In [1]:
DATA_DIR = '/d2/caches/kaggle-planet'
DATA_DIR_TRAIN = DATA_DIR + '/train-tif-v2'
DATA_DIR_TEST = DATA_DIR + '/test-tif-v2'
OUT_DIR = '/d2/caches/kaggle-planet/processed-images-128x128'

In [2]:
%run 'Lib.ipynb'

In [3]:
train_paths = [DATA_DIR_TRAIN + '/' + x for x in list_files(DATA_DIR_TRAIN)]
test_paths = [DATA_DIR_TEST + '/' + x for x in list_files(DATA_DIR_TEST)]
all_paths = train_paths + test_paths

In [4]:
len(all_paths)

101670

In [5]:
SCALE = 0.5

import scipy.ndimage

def process_geotiff(file_path, scale=SCALE):
    
    # load means & stds
    mean_r = np.load('./out/mean_r_sub.npy')
    std_r = np.load('./out/std_r_sub.npy').tolist()
    mean_g = np.load('./out/mean_g_sub.npy')
    std_g = np.load('./out/std_g_sub.npy').tolist()
    mean_b = np.load('./out/mean_b_sub.npy')
    std_b = np.load('./out/std_b_sub.npy').tolist()
    mean_nir = np.load('./out/mean_nir_sub.npy')
    std_nir = np.load('./out/std_nir_sub.npy').tolist()
    
    (fid, _) = os.path.basename(file_path).split('.')
    d = read_geotiff(file_path)
    
    r = (d[:, :, 0] - mean_r) / std_r
    g = (d[:, :, 1] - mean_g) / std_g
    b = (d[:, :, 2] - mean_b) / std_b
    nir = (d[:,:,3] - mean_nir) / std_nir
    
    if 1. != scale:
        r = scipy.ndimage.zoom(r, scale, order=3)
        g = scipy.ndimage.zoom(g, scale, order=3)
        b = scipy.ndimage.zoom(b, scale, order=3)
        nir = scipy.ndimage.zoom(nir, scale, order=3)
    
    img = np.dstack((r, g, b, nir)).astype(np.float32)
    img.tofile(OUT_DIR + '/' + fid)

In [ ]:
import multiprocessing

pool = multiprocessing.Pool(8)
pool.map(process_geotiff, all_paths)
pool.terminate()
pool.close()